### Case a ser respondindo sobre análise de cpf, números e nomes

Suponha que você esteja participando de um projeto voltado à mitigação de fraudes,
e para tal, você deve analisar dados diversos, tais como o CPF, Telefone Celular
(Com DDD incluso) e entre outros. Com isso em mente, responda às seguintes
questões:



a) Quais critérios você utilizaria para indicar se o CPF é válido em termos
estruturais? Construa um algoritmo para automatizar esse processo e realize
um teste com os seguintes CPFs, indicando aqueles que são válidos ou não:
- 217.894.500-71
- 112.622.670-05
- 695.226.930-49
- 015.897.070-51
- 311.692.760-06

In [ ]:
import unicodedata
from difflib import SequenceMatcher
import numpy as np
import re


In [1]:
def validar_cpf(cpf):
    # Remover caracteres não numéricos
    cpf = ''.join([c for c in cpf if c.isdigit()])
    
    # Verificar se o CPF tem 11 dígitos 
    if len(cpf) != 11 or cpf == cpf[0] * 11:
        return False
    
    # Calcular o dígito verificador
    def calcular_digito(cpf, peso_inicial):
        soma = 0
        for i in range(peso_inicial - 1):
            soma += int(cpf[i]) * (peso_inicial - i)
        resto = (soma * 10) % 11
        return 0 if resto == 10 else resto
    
    # Calcular o primeiro dígito verificador
    digito1 = calcular_digito(cpf, 10)
    
    # Calcular o segundo dígito verificador
    digito2 = calcular_digito(cpf[:9] + str(digito1), 11)
    
    # Verificar se os dígitos calculados coincidem com os do CPF
    return cpf[-2:] == f"{digito1}{digito2}"

# CPFs para teste
cpfs = [
    "217.894.500-71",
    "112.622.670-05",
    "695.226.930-49",
    "015.897.070-51",
    "311.692.760-06"
]

# Testando os CPFs
resultados = {cpf: validar_cpf(cpf) for cpf in cpfs}
resultados


{'217.894.500-71': True,
 '112.622.670-05': True,
 '695.226.930-49': True,
 '015.897.070-51': False,
 '311.692.760-06': False}

b) Quais critérios você utilizaria para indicar se um telefone celular é válido ou
não em termos estruturais? Construa um algoritmo para automatizar esse
processo e realize um teste com os seguintes telefones, indicando aqueles
que são válidos ou não:
- (64) 96743-8065
- (75) 89555-5682
- (99) 98362-0347
- (10) 98595-1389

Para verificar se um telefone celular brasileiro é válido em termos estruturais, alguns critérios podem ser utilizados:

Critérios para validar um telefone celular no Brasil:
- DDD válido: O DDD (código de área) deve ser composto por dois dígitos e corresponder a uma região válida no Brasil. Os DDDs válidos estão na faixa de 11 a 99, e "10" não é um DDD válido.
- Primeiro dígito do número: No Brasil, números de celular devem começar com o dígito 9.
- Quantidade de dígitos: O número de telefone, excluindo o DDD, deve ter 9 dígitos.
- Faixa do segundo dígito: O segundo dígito de números de celular geralmente está na faixa de 6 a 9 (mas isso pode variar regionalmente).
- Com base nesses critérios, vamos construir um algoritmo para validar telefones celulares.

In [2]:
def validar_telefone(telefone):
    # Remover caracteres não numéricos
    telefone = ''.join([c for c in telefone if c.isdigit()])
    
    # Verificar se o telefone tem 11 dígitos (2 para o DDD + 9 para o número)
    if len(telefone) != 11:
        return False
    
    # Extrair o DDD e o número
    ddd = telefone[:2]
    numero = telefone[2:]
    
    # Verificar se o DDD está na faixa válida
    if not (11 <= int(ddd) <= 99):
        return False
    
    # Verificar se o número começa com 9 e o segundo dígito está na faixa de 6 a 9
    if numero[0] != '9' or not (6 <= int(numero[1]) <= 9):
        return False
    
    return True

# Telefones para teste
telefones = [
    "(64) 96743-8065",
    "(75) 89555-5682",
    "(99) 98362-0347",
    "(10) 98595-1389"
]

# Testando os telefones
resultados_telefones = {telefone: validar_telefone(telefone) for telefone in telefones}
resultados_telefones


{'(64) 96743-8065': True,
 '(75) 89555-5682': False,
 '(99) 98362-0347': True,
 '(10) 98595-1389': False}

c) Erros e variações de escrita são comuns em processos de preenchimento de
dados como nomes. Com isso em mente, qual estratégia você tomaria para
que em um contexto de avaliação, um modelo retornasse True para as
seguintes comparações:
- Fernando dos Santos / Fernando Santos
- Maria da Silva Rodrigues / Maria Rodrigues
- João Roberto / joao robert


In [3]:

# Função para normalizar os nomes (remover acentos e palavras comuns)
def normalizar_nome(nome):
  
    nome = ''.join(c for c in unicodedata.normalize('NFD', nome) if unicodedata.category(c) != 'Mn')
    
    # Converter para minúsculas e remover palavras comuns
    palavras_comuns = {'da', 'de', 'do', 'dos', 'das'}
    palavras = nome.lower().split()
    palavras_filtradas = [p for p in palavras if p not in palavras_comuns]
    
    return ' '.join(palavras_filtradas)

# Função para comparar nomes com base na similaridade
def comparar_nomes(nome1, nome2, threshold=0.8):
  
    nome1_norm = normalizar_nome(nome1)
    nome2_norm = normalizar_nome(nome2)
    similaridade = SequenceMatcher(None, nome1_norm, nome2_norm).ratio()
    
    return similaridade >= threshold

# Testando os pares de nomes
nomes = [
    ("Fernando dos Santos", "Fernando Santos"),
    ("Maria da Silva Rodrigues", "Maria Rodrigues"),
    ("João Roberto", "joao robert")
]

# Comparando os nomes
resultados_nomes = {f"{nome1} / {nome2}": comparar_nomes(nome1, nome2) for nome1, nome2 in nomes}
resultados_nomes


{'Fernando dos Santos / Fernando Santos': True,
 'Maria da Silva Rodrigues / Maria Rodrigues': True,
 'João Roberto / joao robert': True}